In [1]:
import torch
from knn_loss import KNNLoss

In [2]:
tensor_a = torch.tensor([[2.0], [3.0], [4.0]])
tensor_b = torch.tensor([[0.0], [0.0], [0.0]])
tensor_c = torch.tensor([[-2.0], [-3.0], [-4.0]])

In [3]:
tensors = [tensor_a, tensor_b, tensor_c]
stack = torch.stack(tensors)
torch.mean(stack, dim = 0)
# stack

tensor([[0.],
        [0.],
        [0.]])

In [4]:
forward_pass = torch.randn(256, 512, 1)
labels = torch.rand(256, 1, 1)
labels *= 9
labels = torch.absolute(labels)
labels = torch.round(labels)

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
classes_tensor = torch.tensor(classes)
classes_tensor.shape

torch.Size([10])

In [5]:
wrap = KNNLoss(classes_tensor)
div_loss = wrap.divergence_loss(forward_pass, labels)
print(div_loss)
centroids = wrap.get_centroids()
print(centroids.shape)
conv_loss = wrap.convergence_loss(forward_pass, labels)
print(conv_loss)


0.8283096551895142
torch.Size([10, 512, 1])
22.119428634643555


the `# Calculate Current Centroids` section seem to work.

In [6]:
# a = torch.empty(512, 1)
# b = torch.rand(512, 1)
# c = torch.rand(512, 1)
# d = torch.stack((a, b, c))
# e = d[1:]
# print(d)
# print(e)